## Make predictions on upcoming Tournament

In [14]:
import pandas as pd
import numpy as np

from config import majors, metrics, feat_cols, target_col
from utils import create_rolling_agg_features_by_golfer

import pickle

In [20]:
event_df = pd.read_csv('../data/historical_event_data.csv')
year = 2021
event_name = 'bmw_championship'
pre_tourney = pd.read_csv(f'../data/{year}_{event_name}_pre_tourney_snapshot.csv')
pre_tourney.rename(columns={'bet365':'close_odds'}, inplace=True)

new_event_df = event_df.append(pre_tourney[['dg_id', 'player_name','event_name','close_odds']], sort=False)
new_event_df['major'] = np.where(new_event_df.event_name.isin(majors), 1, 0)

for metric in metrics:
    field = metric.get('field')
    agg = metric.get('agg')
    for period in metric.get('periods'):
        col = f"{field}_in_prev_{period}_events"
        new_event_df[col] = create_rolling_agg_features_by_golfer(new_event_df, field, 1, period, agg)
        
new_event_df = new_event_df.loc[pd.isnull(new_event_df.fin_num)]
X_te = new_event_df[feat_cols]


rf = pickle.load(open("../models/rf_model.pkl", "rb"))

new_event_df['sg_preds'] = rf.predict(X_te)

final_df = pre_tourney.merge(new_event_df[['dg_id','sg_preds']], how='left', on='dg_id')

In [ ]:
#Strokes gained predictions relative to DraftKings Salary

In [51]:
final_df['dk_salary'] = final_df['dk_salary'].astype(int)
final_df['dk_salary_rnk'] = final_df['dk_salary'].rank(ascending=False)
final_df['sg_preds_rnk'] = final_df['sg_preds'].rank(ascending=False)
final_df['rnk_diff'] = final_df['dk_salary_rnk'] - final_df['sg_preds_rnk']

(final_df[['dg_id', 'player_name', 'dk_salary','sg_preds',
           'dk_salary_rnk', 'sg_preds_rnk','rnk_diff']]
 .sort_values('rnk_diff', ascending=False)
 .reset_index(drop=True)
)

,dg_id,player_name,dk_salary,sg_preds,dk_salary_rnk,sg_preds_rnk,rnk_diff
0,12808,"Grillo, Emiliano",6500,0.972802,53.5,44.0,9.5
1,5768,"Hoffman, Charley",7000,1.173585,43.5,34.0,9.5
2,7108,"Casey, Paul",8200,3.317274,23.0,15.0,8.0
3,19841,"DeChambeau, Bryson",9300,4.183491,12.0,6.0,6.0
4,13900,"Lowry, Shane",7900,3.133729,26.0,20.0,6.0
...,...,...,...,...,...,...,...
64,7649,"Leishman, Marc",6700,0.508251,49.5,57.0,-7.5
65,14735,"Ortiz, Carlos",6600,0.384003,51.5,59.0,-7.5
66,17550,"Van Rooyen, Erik",7500,1.013864,33.5,41.0,-7.5
67,18841,"Hovland, Viktor",9100,3.090208,14.0,22.0,-8.0
